# Projects & Properties & Context Management

Nexus provides several ways to organize your data, store custom parameters and integrate these easily into your workflows.

- Projects can be thought of as a 'virtual container' for your workflows and data.

- Properties are a place to store additional custom metadata or scientific parameters. Properties are defined on a Project, and given values for data within that project.


In [1]:
from datetime import datetime
import qnexus as qnx

from pytket import Circuit

In [2]:
# Get a ProjectRef 
my_project_ref = qnx.projects.create(name=f"My Project from {datetime.now()}")

# Show basic information about the project
my_project_ref.df()

,name,description,created,modified,contents_modified,id
0,My Nexus Project from 2024-07-18 18:39:04.851887,None,2024-07-18 17:39:06.042915+00:00,2024-07-18 17:39:06.042915+00:00,2024-07-18 17:39:06.045780+00:00,51c3d3a0-9540-460a-819b-3e7c489cae56


In [3]:
# Get a summary of the project as a DataFrame (shows Job statuses)
qnx.projects.summarize(my_project_ref)

,project,total_jobs,pending_jobs,cancelled_jobs,errored_jobs,completed_jobs
0,My Nexus Project from 2024-07-18 18:39:04.851887,0,0,0,0,0


In [4]:
# Lets add a custom property 

qnx.projects.add_property(
    name="molecule_type",
    property_type="string",
    project=my_project_ref,
    description="The type of molecule being studied",
)

qnx.projects.add_property(
    name="iteration",
    property_type="int",
    project=my_project_ref,
    description="The iteration number of the experiment",
)

In [5]:
# View the properties defined on the project as a DataFrame

qnx.projects.get_properties(project=my_project_ref).df()

,name,description,created,modified,property_type,required,color
0,molecule_type,The type of molecule being studied,2024-07-18 17:39:06.570100+00:00,2024-07-18 17:39:06.570100+00:00,string,False,#d55e00
1,iteration,The iteration number of the experiment,2024-07-18 17:39:06.757608+00:00,2024-07-18 17:39:06.757608+00:00,int,False,#cc79a7


In [6]:
# We can then define property values on data we create within the project

my_circuit_ref = qnx.circuits.upload(
    name=f"My Circuit from {datetime.now()}",
    circuit = Circuit(2).ZZPhase(0.5, 0, 1).measure_all(),
    project = my_project_ref,
    properties={"molecule_type": "H2"},
)

my_circuit_ref.df()

,name,description,created,modified,molecule_type,project,id
0,My Circuit from 2024-07-18 18:39:07.851679,None,2024-07-18 17:39:08.040672+00:00,2024-07-18 17:39:08.040672+00:00,H2,My Nexus Project from 2024-07-18 18:39:04.851887,b1587e3d-f522-4145-89eb-11013ef970e8


If you submit a Job to Nexus with defined property values, these values will be propogated to the outputs of the Job.

## Context Management

If we know we'll be working within a project or with a set of defined property values we can provide these in a system of context management.

In [8]:
# Via a typical context manager
with qnx.context.using_project(my_project_ref):
    # Show all circuits  in the project
    qnx.circuits.get_all().df()

    # These values can be overridden, with parameters taking precedence over context
    my_circuit_ref = qnx.circuits.upload(
        name=f"My Circuit from {datetime.now()}",
        circuit = Circuit(2).ZZPhase(0.5, 0, 1).measure_all(),
        project=qnx.projects.create(name=f"My other Nexus Project from {datetime.now()}")
    )

    with qnx.context.using_properties(molecule_type = "H2", iteration = 0):

        # Create a circuit within the project and with defined property values
        my_circuit_ref = qnx.circuits.upload(
            name=f"My Circuit from {datetime.now()}",
            circuit = Circuit(2).ZZPhase(0.5, 0, 1).measure_all(),
        )

        # Property parameters will be the union of the parameters and context (parameters take precedence)
        my_circuit_ref = qnx.circuits.upload(
            name=f"My Circuit from {datetime.now()}",
            circuit = Circuit(2).ZZPhase(0.5, 0, 1).measure_all(),
            properties={"iteration": 1},
        )


In [9]:
# Values can also be set in the global context

current_project_context = qnx.context.set_active_project_token(my_project_ref)

# Deactivate the project context

qnx.context.deactivate_project(current_project_context)
